In [1]:
import pandas as pd

In [2]:
df_list = []

for i in range(12):
    df_list.append(
        pd.read_csv(f"columns/out_{i}.csv", sep="⇧", engine="python", quotechar='"', on_bad_lines="warn")
    )
df = pd.concat(df_list, axis=0)

Skipping line 48859: '⇧' expected after '"'
Skipping line 48860: '⇧' expected after '"'
Skipping line 48861: '⇧' expected after '"'
Skipping line 344849: '⇧' expected after '"'


In [3]:
df

,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0_meta.json,0,Чарт (2019)
1,dataset/Dataset/data/7779522/table_0_meta.json,1,Высшаяпозиция
2,dataset/Dataset/data/2863377/table_11_meta.json,0,Unnamed: 0
3,dataset/Dataset/data/2863377/table_11_meta.json,1,Команда
4,dataset/Dataset/data/2863377/table_11_meta.json,2,И
...,...,...,...
414413,dataset/Dataset/data/35185/table_8_meta.json,2,Дата получения Любекского права
414414,dataset/Dataset/data/35185/table_8_meta.json,3,Дата получения Кульмского права
414415,dataset/Dataset/data/35185/table_8_meta.json,4,Дата получения Магдебургского права
414416,dataset/Dataset/data/35185/table_8_meta.json,5,Дата вступления в союз


# Research

Проблемные столбцы (скорее всего не будем использовать):
- Столбцы по типу: "ΔHкип, кДж·моль−1"
- Столбцы использующиеся для группировки каких-либо значений по времени (месяцу, году и т.п.)
- Столбцы без имени
- Столбцы, название которых не имеет смысла (числа и т.п.)
- Аббревиатуры и сокращения

### Столбцы без названия = 164500
их можно отправить в тестовую выборку, чтобы не терять информацию. (нужно еще посмотреть, что внутри них обычно лежит)

### Без имени
можно в тест

## .mv-parser... = 15661
обрезать только осмысленный текст

---

In [4]:
df["column_name"].value_counts()

column_name
Год                                                           220712
Место                                                          87208
Роль                                                           78995
Команда                                                        67732
Название                                                       64980
                                                               ...  
Аффилированность                                                   1
Сколько получил штрафных баллов за нарушения игры                  1
Сколько очков принёс команде в результате заброшенных шайб         1
Совершил передачи                                                  1
Дата постановления                                                 1
Name: count, Length: 125704, dtype: int64

## Столбцы без названия

In [5]:
unnamed_columns = df["column_name"].apply(lambda x: str(x).startswith("Unnamed"))
df[unnamed_columns]

,table_id,column_id,column_name
2,dataset/Dataset/data/2863377/table_11_meta.json,0,Unnamed: 0
13,dataset/Dataset/data/4727026/table_0_meta.json,1,Unnamed: 1
17,dataset/Dataset/data/5212502/table_3_meta.json,0,Unnamed: 0
35,dataset/Dataset/data/6995744/table_0_meta.json,1,Unnamed: 1
257,dataset/Dataset/data/632307/table_0_meta.json,5,Unnamed: 5
...,...,...,...
414064,dataset/Dataset/data/8352540/table_6_meta.json,1,Unnamed: 1
414066,dataset/Dataset/data/2239378/table_1_meta.json,0,Unnamed: 0
414166,dataset/Dataset/data/529438/table_0_meta.json,1,Unnamed: 1
414234,dataset/Dataset/data/6399252/table_2_meta.json,13,Unnamed: 13


## Столбцы содержащие в названии: ".mw-parser..."

In [6]:
mv_parser = df["column_name"].apply(lambda x: str(x).strip().startswith(".mw-parser"))
df[mv_parser]

,table_id,column_id,column_name
63,dataset/Dataset/data/6573775/table_0_meta.json,16,.mw-parser-output .ts-comment-commentedText{bo...
426,dataset/Dataset/data/1793136/table_0_meta.json,5,.mw-parser-output .ts-comment-commentedText{bo...
610,dataset/Dataset/data/6558206/table_1_meta.json,0,.mw-parser-output .ts-comment-commentedText{bo...
757,dataset/Dataset/data/800584/table_2_meta.json,0,.mw-parser-output .ts-comment-commentedText{bo...
889,dataset/Dataset/data/5446712/table_1_meta.json,0,.mw-parser-output .ts-comment-commentedText{bo...
...,...,...,...
413650,dataset/Dataset/data/5375601/table_1_meta.json,0,.mw-parser-output .ts-comment-commentedText{bo...
413973,dataset/Dataset/data/6457969/table_1_meta.json,0,.mw-parser-output .ts-comment-commentedText{bo...
414108,dataset/Dataset/data/8273109/table_5_meta.json,0,.mw-parser-output .ts-comment-commentedText{bo...
414196,dataset/Dataset/data/6502130/table_2_meta.json,9,.mw-parser-output .ts-comment-commentedText{bo...


### В назнваниях столбца содержится человекопонятное название, нужно отчистить мусорную информацию

In [7]:
mv_parser_humnareadable_names = df[mv_parser]["column_name"].apply(lambda x: str(x).split("}")[-1])
mv_parser_humnareadable_names.unique()

array(['Поз.', 'И', 'М', '№', 'Pos.', 'В', 'Пр.', 'П.', ' В', 'Категория',
       'Коэфф.', 'Пропущено', '03/04', 'Жертвы', 'И.', '#', 'П',
       'Оценка D', 'No.', 'Σ', 'СКР (общий)', 'Прим.', 'ЖК', 'ХП', 'Поз',
       '\xa0В', '№ п/п', 'Лига', 'Ист.', 'SP', 'SP.1', 'СС', 'КН',
       'Кубок', 'Дата рождения', 'ТТХ', 'Древн.', 'PPG', 'СЭ', 'А',
       'Участия в Рестлманиях', 'СУ', 'Уров.', 'MVP', 'З',
       'Должность / Титул', '№№', 'ГП[69]', 'Л/круги',
       'Медиафайлы на\xa0ВикискладеМодель', 'Место', 'Пал.', 'ЧР',
       'Всего депутатов', 'ГП[67]', 'Д / В', 'Б/Н', 'С', 'В–П', 'И, С',
       ' И', 'Кат.', 'Субъект РФ', 'Звание', 'КИ', 'Рез.',
       'Количество платных показов (млн)', '12/13', 'Л/Круги', 'Ист',
       'д/г', 'МГ', 'Дата', 'HP', 'Гр.', 'RPG', 'Сезоны', 'Л.круги',
       'Конф.', 'Сн.', '+/- (в\xa0%)', 'Быстрый круг',
       'Плотность(чел./км²)', 'БК', 'ПП', 'Результат', 'Доля, %', 'КП',
       'ТА (БК торпед)', 'Кат', 'Возраст', 'Н', '06/07', '15+15 км',
    

## Аббревиатуры / сокращения / меры (кг, кдж.)

In [8]:
# TODO: what should i do? -_-

# Очистка данных

1. Очистка мусора из столбцов, содержащих ".mv-parser.."
2. Добавление столбца, содержащего первое слово в названии столбца
3. Очистка символов (оставить только буквы алфавита а-яА-Я)
4. Столбцы без названия выделить в тестовую выборку (столбцы для которых нет в vocabulary w2v, тоже в тест)
---

In [9]:
df_cleansed = df.copy()

## 1. Очистка мусора из столбцов

In [10]:
def clean_mv_parser(col_name: str) -> str:
    if col_name.strip().startswith(".mw-parser"):
        return col_name.split("}")[-1]
    return col_name

df_cleansed.loc[:, "column_name"] = df_cleansed["column_name"].apply(lambda x: clean_mv_parser(str(x)))

In [11]:
df_cleansed[mv_parser]
# df_cleansed[~mv_parser]

,table_id,column_id,column_name
63,dataset/Dataset/data/6573775/table_0_meta.json,16,Поз.
426,dataset/Dataset/data/1793136/table_0_meta.json,5,И
610,dataset/Dataset/data/6558206/table_1_meta.json,0,М
757,dataset/Dataset/data/800584/table_2_meta.json,0,М
889,dataset/Dataset/data/5446712/table_1_meta.json,0,М
...,...,...,...
413650,dataset/Dataset/data/5375601/table_1_meta.json,0,М
413973,dataset/Dataset/data/6457969/table_1_meta.json,0,М
414108,dataset/Dataset/data/8273109/table_5_meta.json,0,М
414196,dataset/Dataset/data/6502130/table_2_meta.json,9,APG


## 2. Добавление столбца с первым словом в названии столбца

In [12]:
df_cleansed['first_word'] = df_cleansed["column_name"].apply(lambda x: str(x).split(" ")[0].lower())
df_cleansed["first_word"]

0                  чарт
1         высшаяпозиция
2              unnamed:
3               команда
4                     и
              ...      
414413             дата
414414             дата
414415             дата
414416             дата
414417             дата
Name: first_word, Length: 4968218, dtype: object

## 3. Очистка символов (оставляем только буквы)

In [13]:
def remove_non_alphabetic_symbols(col_name: str) -> str:
    return "".join(c for c in col_name if c.isalpha())

df_cleansed["first_word"] = df_cleansed["first_word"].apply(lambda x: remove_non_alphabetic_symbols(str(x)))
df_cleansed

,table_id,column_id,column_name,first_word
0,dataset/Dataset/data/7779522/table_0_meta.json,0,Чарт (2019),чарт
1,dataset/Dataset/data/7779522/table_0_meta.json,1,Высшаяпозиция,высшаяпозиция
2,dataset/Dataset/data/2863377/table_11_meta.json,0,Unnamed: 0,unnamed
3,dataset/Dataset/data/2863377/table_11_meta.json,1,Команда,команда
4,dataset/Dataset/data/2863377/table_11_meta.json,2,И,и
...,...,...,...,...
414413,dataset/Dataset/data/35185/table_8_meta.json,2,Дата получения Любекского права,дата
414414,dataset/Dataset/data/35185/table_8_meta.json,3,Дата получения Кульмского права,дата
414415,dataset/Dataset/data/35185/table_8_meta.json,4,Дата получения Магдебургского права,дата
414416,dataset/Dataset/data/35185/table_8_meta.json,5,Дата вступления в союз,дата


## 4. Выделение столбцов без названия

In [14]:
unnamed_columns = df_cleansed["first_word"] == "unnamed"
df_cleansed[unnamed_columns]

,table_id,column_id,column_name,first_word
2,dataset/Dataset/data/2863377/table_11_meta.json,0,Unnamed: 0,unnamed
13,dataset/Dataset/data/4727026/table_0_meta.json,1,Unnamed: 1,unnamed
17,dataset/Dataset/data/5212502/table_3_meta.json,0,Unnamed: 0,unnamed
35,dataset/Dataset/data/6995744/table_0_meta.json,1,Unnamed: 1,unnamed
257,dataset/Dataset/data/632307/table_0_meta.json,5,Unnamed: 5,unnamed
...,...,...,...,...
414064,dataset/Dataset/data/8352540/table_6_meta.json,1,Unnamed: 1,unnamed
414066,dataset/Dataset/data/2239378/table_1_meta.json,0,Unnamed: 0,unnamed
414166,dataset/Dataset/data/529438/table_0_meta.json,1,Unnamed: 1,unnamed
414234,dataset/Dataset/data/6399252/table_2_meta.json,13,Unnamed: 13,unnamed


In [15]:
df_cleansed = df_cleansed[~unnamed_columns]
df_cleansed

,table_id,column_id,column_name,first_word
0,dataset/Dataset/data/7779522/table_0_meta.json,0,Чарт (2019),чарт
1,dataset/Dataset/data/7779522/table_0_meta.json,1,Высшаяпозиция,высшаяпозиция
3,dataset/Dataset/data/2863377/table_11_meta.json,1,Команда,команда
4,dataset/Dataset/data/2863377/table_11_meta.json,2,И,и
5,dataset/Dataset/data/2863377/table_11_meta.json,3,В,в
...,...,...,...,...
414413,dataset/Dataset/data/35185/table_8_meta.json,2,Дата получения Любекского права,дата
414414,dataset/Dataset/data/35185/table_8_meta.json,3,Дата получения Кульмского права,дата
414415,dataset/Dataset/data/35185/table_8_meta.json,4,Дата получения Магдебургского права,дата
414416,dataset/Dataset/data/35185/table_8_meta.json,5,Дата вступления в союз,дата


# Лемматизация (оооочень медленно, не делаю, пока не найду способ быстрее)

In [16]:
import spacy
nlp = spacy.load("ru_core_news_sm", disable=['parser', 'ner'])

In [17]:
doc = nlp("Даты")
doc[0].lemma_


'дата'

In [18]:
""" Too slow
def lemmatize_first_word(first_word: str) -> str:
    try:
        return nlp(first_word)[0].lemma_
    except IndexError:
        return first_word

df_cleansed["first_word"].apply(lemmatize_first_word)
"""

' Too slow\ndef lemmatize_first_word(first_word: str) -> str:\n    try:\n        return nlp(first_word)[0].lemma_\n    except IndexError:\n        return first_word\n\ndf_cleansed["first_word"].apply(lemmatize_first_word)\n'

# Создание ембеддингов

Для кластеризации удаляю столбцы, для которых не было получено ембеддингов

In [19]:
import gensim.downloader

ru_model = gensim.downloader.load("word2vec-ruscorpora-300")

In [20]:
def get_embedding(word: str) -> list|None:
    try:
        return ru_model.__getitem__([f"{word}_NOUN"])
    except KeyError:
        return None

df_cleansed["embedding"] = df_cleansed["first_word"].apply(get_embedding)
df_cleansed

/tmp/ipykernel_39686/1837533370.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleansed["embedding"] = df_cleansed["first_word"].apply(get_embedding)


,table_id,column_id,column_name,first_word,embedding
0,dataset/Dataset/data/7779522/table_0_meta.json,0,Чарт (2019),чарт,"[[0.029721806, 0.03733756, -0.05228397, 0.0204..."
1,dataset/Dataset/data/7779522/table_0_meta.json,1,Высшаяпозиция,высшаяпозиция,None
3,dataset/Dataset/data/2863377/table_11_meta.json,1,Команда,команда,"[[0.05975376, -0.050061475, 0.025441337, 0.015..."
4,dataset/Dataset/data/2863377/table_11_meta.json,2,И,и,None
5,dataset/Dataset/data/2863377/table_11_meta.json,3,В,в,None
...,...,...,...,...,...
414413,dataset/Dataset/data/35185/table_8_meta.json,2,Дата получения Любекского права,дата,"[[0.11930524, 0.021293541, -0.06631852, -0.068..."
414414,dataset/Dataset/data/35185/table_8_meta.json,3,Дата получения Кульмского права,дата,"[[0.11930524, 0.021293541, -0.06631852, -0.068..."
414415,dataset/Dataset/data/35185/table_8_meta.json,4,Дата получения Магдебургского права,дата,"[[0.11930524, 0.021293541, -0.06631852, -0.068..."
414416,dataset/Dataset/data/35185/table_8_meta.json,5,Дата вступления в союз,дата,"[[0.11930524, 0.021293541, -0.06631852, -0.068..."


In [21]:
df_prepared_for_clustering = df_cleansed[~df_cleansed['embedding'].isna()]
df_prepared_for_clustering

,table_id,column_id,column_name,first_word,embedding
0,dataset/Dataset/data/7779522/table_0_meta.json,0,Чарт (2019),чарт,"[[0.029721806, 0.03733756, -0.05228397, 0.0204..."
3,dataset/Dataset/data/2863377/table_11_meta.json,1,Команда,команда,"[[0.05975376, -0.050061475, 0.025441337, 0.015..."
8,dataset/Dataset/data/2863377/table_11_meta.json,6,ГЗ,гз,"[[-0.0009766041, 0.053358104, 0.020000864, 0.0..."
9,dataset/Dataset/data/2863377/table_11_meta.json,7,ГП,гп,"[[-0.0041590976, 0.0012636776, -0.05824213, 0...."
11,dataset/Dataset/data/2863377/table_11_meta.json,9,Очки,очки,"[[0.029941676, -0.03986928, 0.060400773, 0.034..."
...,...,...,...,...,...
414413,dataset/Dataset/data/35185/table_8_meta.json,2,Дата получения Любекского права,дата,"[[0.11930524, 0.021293541, -0.06631852, -0.068..."
414414,dataset/Dataset/data/35185/table_8_meta.json,3,Дата получения Кульмского права,дата,"[[0.11930524, 0.021293541, -0.06631852, -0.068..."
414415,dataset/Dataset/data/35185/table_8_meta.json,4,Дата получения Магдебургского права,дата,"[[0.11930524, 0.021293541, -0.06631852, -0.068..."
414416,dataset/Dataset/data/35185/table_8_meta.json,5,Дата вступления в союз,дата,"[[0.11930524, 0.021293541, -0.06631852, -0.068..."


In [22]:
embeddings = df_prepared_for_clustering['embedding']
embeddings

0         [[0.029721806, 0.03733756, -0.05228397, 0.0204...
3         [[0.05975376, -0.050061475, 0.025441337, 0.015...
8         [[-0.0009766041, 0.053358104, 0.020000864, 0.0...
9         [[-0.0041590976, 0.0012636776, -0.05824213, 0....
11        [[0.029941676, -0.03986928, 0.060400773, 0.034...
                                ...                        
414413    [[0.11930524, 0.021293541, -0.06631852, -0.068...
414414    [[0.11930524, 0.021293541, -0.06631852, -0.068...
414415    [[0.11930524, 0.021293541, -0.06631852, -0.068...
414416    [[0.11930524, 0.021293541, -0.06631852, -0.068...
414417    [[0.11930524, 0.021293541, -0.06631852, -0.068...
Name: embedding, Length: 2535388, dtype: object

### Столбцы, для которых не построились эмбеддинги

In [23]:
df_cleansed[df_cleansed['embedding'].isna()]

,table_id,column_id,column_name,first_word,embedding
1,dataset/Dataset/data/7779522/table_0_meta.json,1,Высшаяпозиция,высшаяпозиция,None
4,dataset/Dataset/data/2863377/table_11_meta.json,2,И,и,None
5,dataset/Dataset/data/2863377/table_11_meta.json,3,В,в,None
6,dataset/Dataset/data/2863377/table_11_meta.json,4,Н,н,None
7,dataset/Dataset/data/2863377/table_11_meta.json,5,П,п,None
...,...,...,...,...,...
414404,dataset/Dataset/data/6418067/table_1_meta.json,3,В,в,None
414405,dataset/Dataset/data/6418067/table_1_meta.json,4,Н,н,None
414406,dataset/Dataset/data/6418067/table_1_meta.json,5,П,п,None
414409,dataset/Dataset/data/6418067/table_1_meta.json,8,+/-,,None


# Кластеризация

- DBScan O(n^2) + съедает ресурсы, не подходит для датасета.
- Использую mini batch k-means

Можно улучшить, взяв параллельный k-means на cuda https://github.com/src-d/kmcuda

In [24]:
from sklearn.cluster import MiniBatchKMeans
import numpy as np

In [25]:
embeddings_reshaped = embeddings.to_numpy()
embeddings_reshaped = np.array(list(map(lambda x: x[0], embeddings_reshaped)))
embeddings_reshaped.shape

(2535388, 300)

In [26]:
num_clusters = 130
kmeans = MiniBatchKMeans(n_clusters=num_clusters, random_state=13, max_iter=200)

In [27]:
df_prepared_for_clustering["cls"] = kmeans.fit_predict(embeddings_reshaped)

/home/kirilltobola/Dev/github.com/isp/cta/venv/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/tmp/ipykernel_39686/40741157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prepared_for_clustering["cls"] = kmeans.fit_predict(embeddings_reshaped)


In [34]:
from random import randint

df_prepared_for_clustering[df_prepared_for_clustering['cls'] == randint(0, num_clusters - 1)]

,table_id,column_id,column_name,first_word,embedding,cls
4182,dataset/Dataset/data/5804672/table_1_meta.json,4,Ранг,ранг,"[[0.015170229, -0.09289474, 0.0062569017, 0.14...",113
7467,dataset/Dataset/data/6720575/table_1_meta.json,1,Ранг,ранг,"[[0.015170229, -0.09289474, 0.0062569017, 0.14...",113
7693,dataset/Dataset/data/7702638/table_1_meta.json,2,Ранг,ранг,"[[0.015170229, -0.09289474, 0.0062569017, 0.14...",113
8705,dataset/Dataset/data/7167902/table_2_meta.json,0,Ранг,ранг,"[[0.015170229, -0.09289474, 0.0062569017, 0.14...",113
10661,dataset/Dataset/data/4935364/table_12_meta.json,2,Капитан,капитан,"[[0.031978782, -0.09379112, -0.0026279353, 0.0...",113
...,...,...,...,...,...,...
407444,dataset/Dataset/data/694098/table_16_meta.json,1,Капитан,капитан,"[[0.031978782, -0.09379112, -0.0026279353, 0.0...",113
408424,dataset/Dataset/data/4109401/table_4_meta.json,2,Капитан,капитан,"[[0.031978782, -0.09379112, -0.0026279353, 0.0...",113
410046,dataset/Dataset/data/4126117/table_3_meta.json,2,Ранг,ранг,"[[0.015170229, -0.09289474, 0.0062569017, 0.14...",113
410274,dataset/Dataset/data/6502339/table_1_meta.json,3,Ранг,ранг,"[[0.015170229, -0.09289474, 0.0062569017, 0.14...",113


In [36]:
df_prepared_for_clustering[df_prepared_for_clustering['cls'] == randint(0, num_clusters - 1)]["first_word"].unique()

array(['население', 'народонаселение'], dtype=object)